# ДЗ 1

## Задание 1

Сравните метрики hit_rate@k, precision@k. Какую матрику использовать предпочтительно и почему. Привидите пример 2-3 задач (опишите, что является клиентом, что товаром), в которой более уместно использовать метрику hit_rate?

### Ответ

hit_rate@k - это наличие хотя бы одного релевантного товара из k рекомендованных. <br>precision@k - это доля релевантных товаров из k рекомендованных. 
- Рекомендации в on-line кинотеатре, нас будет интересовать доля релевантных - precision@k
- Рекомендации кандидатов (резюме) на закрытие "не массовой" вакансии (например для ТОП-1)- hit_rate@k

## Задание 2

В метрике NDCG@k мы используем логарифм в знаменателе. Как Вы думаете, почему именно логарифм? Какую функцию можно использовать вместо логарифма? Привидите пример метрик/подходов к предобработке данных/функций ошибок в ML, где также в знаменателе присутствует логарифм.

### Ответ

Логарифм исполтзуется для "сглаживания" + удобано использовать при дифференцировании. 

## Задание 3

Какие еще метрики (Вы можете вспомнить уже пройденные Вами или посмотреть в интернете) могут использоваться для рекомендательных систем (приведите примеры метрики и чем являются интеракции, чтобы она могла быть использована).

### Ответ

#### Непрерывные метрики:
Оценка таргетов (товаров, e-товар, ресюме и т.п.) 
- MAE (Mean Absolute Error) Среднее абсолютное отклонение
- MSE (Mean Squared Error) Среднеквадратичная ошибка
- RMSE (Root Mean Squared Error) Корень из среднеквадратичной ошибки

#### Дискретные метрики:
Используются при учёте взаимодействий (движение пользователя/клиента по воронке продажи и взаимодействия):
- Precision Доля рекомендаций, понравившихся пользователю
- Recall Доля интересных пользователю товаров, которая показана
- F1-Measure Среднее гармоническое метрик Precision и Recall. Полезно, когда заранее невозможно сказать, какая из метрик важнее
- ROC AUC Насколько высока концентрация интересных товаров в начале списка рекомендаций
- Precision@N Метрика Precision, посчитанная на Top-N записях
- Recall@N Метрика Recall, посчитанная на Top-N записях
- AverageP Среднее значение Precision на всем списке рекомендаций

#### Метрики ранжирования (Rank Accuracy):
Оценки релевантности в зависимости от позиции
- Mean Reciprocal Rank На какой позиции списка рекомендаций пользователь находит первую полезную
- Spearman Correlation Корреляция (Спирмена) реального и прогнозируемого рангов рекомендаций
- nDCG Информативность выдачи с учетом ранжирования рекомендаций
- Fraction of Concordance Pairs Насколько высока концентрация интересных товаров в начале списка рекомендаций

## Задание 4

In [2]:
boughted = [1, 3, 5, 7, 9, 11]
recommended = [2, 5, 7, 4, 11, 9, 8, 10, 12, 3]
k = 8

Посчитайте на этих данных pr@8, rec@8, AP@8, NDCG@8, RR@8, ERR@8

In [3]:
import numpy as np

### pr@8

In [12]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    print(flags)
    
    precision = flags.sum() / len(recommended_list)
    
    return precision

In [13]:
precision_at_k(recommended, boughted, k)

[False False  True  True  True  True]


0.5

### rec@8

In [14]:
def recall_at_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    recall = flags.sum() / len(bought_list)
    
    return recall

In [15]:
recall_at_k(recommended, boughted, k)

0.6666666666666666

### AP@8

In [26]:
def ap_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(recommended_list, bought_list)
    
    if sum(flags) == 0:
        return 0
    
    sum_ = 0
    for i in range(1, k+1): 
        if flags[i] == True:
            p_k = precision_at_k(recommended_list, bought_list, k=i)
            sum_ += p_k
            
    result = sum_ / sum(flags)
    
    return result

In [27]:
ap_at_k(recommended, boughted, k)

[False False False False False False]
[False False  True False False False]
[False False  True  True False False]
[False False  True  True False  True]


0.32

### NDCG@8

In [63]:
[1/np.log(i) for i in range(2, k+1)]

[1.4426950408889634,
 0.9102392266268373,
 0.7213475204444817,
 0.6213349345596119,
 0.5581106265512472,
 0.5138983423697507,
 0.48089834696298783]

In [64]:
def ndcg_at_k(recommended_list, bought_list, k=8):
    
    if k <= 0:
        return -1
    
    if len(recommended_list) < k:
        k = len(recommended_list)
    
    idcg = sum(1/np.log(i) for i in range(2, k+1))+1
    
    dcg = 0
    if recommended_list[0] in bought_list:
        dcg = 1
    dcg += sum(1/np.log(i) if recommended_list[i-1] in bought_list else 0 for i in range(2, k+1))
    
    return dcg/idcg

In [65]:
ndcg_at_k(recommended, boughted, k)

0.5653142737255068

### RR@8

In [28]:
def reciprocal_rank_at_k(recommended_list, bought_list, k=8):
    recommended_list = recommended_list[:k]
    ranks=0.
    for item_rec in recommended_list:
        for i, item_bought in enumerate(bought_list):
            if item_rec == item_bought:
                ranks += 1 / (i+1)
    return ranks / len(recommended_list)

In [29]:
reciprocal_rank_at_k(recommended, boughted, k)

0.095

### ERR@8

Expected Reciprocal Rank (ERR)

In [59]:
grade = max(recommended)
grade

12

In [60]:
prob = [0]

In [61]:
prob.extend((2**i-1)/2**grade for i in range(1, grade+1))
prob

[0,
 0.000244140625,
 0.000732421875,
 0.001708984375,
 0.003662109375,
 0.007568359375,
 0.015380859375,
 0.031005859375,
 0.062255859375,
 0.124755859375,
 0.249755859375,
 0.499755859375,
 0.999755859375]

In [78]:
[np.prod([j+1 for j in range(i)]) for i in range(1, k+1)]

[1, 2, 6, 24, 120, 720, 5040, 40320]

In [95]:
rec_lst = recommended
prob[rec_lst[0]]

0.000732421875

In [102]:
[1/i*prob[rec_lst[i-1]]*np.prod([1-prob[rec_lst[j-1]] for j in range(1, i)]) for i in range(1, k+1)]

[0.000732421875,
 0.0037814080715179443,
 0.010249552797176873,
 0.0008797815889804994,
 0.09569694787104713,
 0.009958683006034747,
 0.003728237850864333,
 0.012272455804730787]

In [105]:
np.prod([1-prob[rec_lst[j-1]] for j in range(1, 1)])

1.0

In [103]:
def expected_reciprocal_rank_at_k(rec_lst, k=8):
    
    if k <= 0:
        return -1
    
    if len(rec_lst) < k:
        k = len(rec_lst)
    
    rec_lst = rec_lst[:k]
    
    grade = max(rec_lst)
    prob = [0]
    prob.extend((2**i-1)/2**grade for i in range(1, grade+1))
    
    return sum(1/i*prob[rec_lst[i-1]]*np.prod([1-prob[rec_lst[j-1]] for j in range(1, i)]) for i in range(1, k+1))

In [104]:
expected_reciprocal_rank_at_k(recommended, k)

0.2141286747780282